In [1]:
import requests
from bs4 import BeautifulSoup
import time
import MySQLdb
import math
import json
import random
import re

In [2]:
# Librarii model
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torchvision.utils as utils
import cv2
import time
import os
import copy
import warnings
import pdb;
import torch.nn.functional as F
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import matplotlib.image as mpimg
from PIL import *

In [3]:
class Attention(nn.Module):
    def __init__(self, in_features):
        super(Attention, self).__init__()
        self.op = nn.Sequential(
                    nn.Conv2d(in_channels=in_features, out_channels=64, kernel_size=3, padding=1),
                    nn.Conv2d(in_channels=64, out_channels=16, kernel_size=3, padding=1),
                    nn.Conv2d(in_channels=16, out_channels=1, kernel_size=1, padding=0))
    def forward(self, l):
        N, C, W, H = l.size()
        c = self.op(l)
        a = torch.sigmoid(c)
        g = torch.mul(a.expand_as(l), l)
        return a.view(N,1,W,H), g

class ModifyVGG(nn.Module):
    def __init__(self, im_size, num_classes):
        super(ModifyVGG,self).__init__()
        vgg_model = models.vgg16(pretrained=True)		
        
        # everything before layer 4
        self.before_att = nn.Sequential(*list(vgg_model.features.children())[:4])
        # added layer
        self.attention1 = Attention(in_features=64)
        self.attention2 = Attention(in_features=128)
        self.attention3 = Attention(in_features=256)
        # everything after layer 4
        self.after_att = nn.Sequential(*list(vgg_model.features.children())[4:]) 
        self.avg_pool = vgg_model.avgpool
        self.classifier = nn.Sequential(*list(vgg_model.classifier.children()))
    def forward(self,x):
      # run vgg before layer 4
      x = self.before_att(x)
      # run added layer
      c1, x = self.attention1(x)
      # run everything after layer 4 (as in the original vgg model)
      x = self.after_att[:5](x)
      c2, x = self.attention2(x)
      x = self.after_att[5:12](x)
      c3, x = self.attention3(x)
      x = self.after_att[12:](x)
      x = self.avg_pool(x)
      x = x.view((x.shape[0],x.shape[1]*x.shape[2]*x.shape[3]))
      x = self.classifier(x)
      return x, c1, c2, c3
IMAGE_LOCATION = 'testing/'
def get_html_source(URL):
    """Docstring here."""
    html_source = requests.get(URL).text
    return BeautifulSoup(html_source, 'html.parser')


def image_name(last_name):
    """Docstring here."""
    return '{}.jpg'.format(os.path.join(IMAGE_LOCATION, str(last_name+1)))


def validate(image_url):
    """Docstring here."""
    return image_url.endswith('.jpg') and image_url.startswith('http')


def download_images(soup, current_name):
    """Docstring here."""
    da = soup.findAll('table','fixed offers breakword redesigned')
    prices, links = [], []
    for p in da[0].findAll('table'):
        for tabel in p.findAll('td','photo-cell'):
            for t in tabel.findChildren("img"):
                image_url = t.get('src')
                image = requests.get(image_url)
                if image.status_code == 200:
                    with open(image_name(current_name), 'wb') as f:
                        f.write(image.content)
        for price in p.findAll('p','price'):
            prices.append(price.get_text().strip()[:5].replace(" ",""))
        for link in p.findAll('a','linkWithHash'):
            links.append(link['href'])
            break
        current_name += 1

    return current_name, prices, links

def get_prices_olx(soup):
    da = soup.findAll('table','fixed offers breakword redesigned')
    prices, links = [], []
    for p in da[0].findAll('table'):
        for price in p.findAll('p','price'):
            prices.append(price.get_text().strip()[:5].replace(" ",""))
            break
        for link in p.findAll('a','linkWithHash'):
            links.append(link['href'])
            break
        
    return prices, links
def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.clone().detach().unsqueeze(0)
    return image

data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [4]:
# EMAG

def search_EMAG(URL):
    url = 'https://www.emag.ro/search/'
    URL = url + URL
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    soup


    nr_pagini = soup.findAll("p", "product-new-price")

    preturi = []
    for pret in nr_pagini:
        if len(pret.text) > 0:
            try:
                preturi.append(int(str(pret).split(">",1)[1].split("<",1)[0].replace('.','')))
            except:
                continue
                
    link = soup.findAll("a", "thumbnail-wrapper js-product-url", href=True)
    links = []
    for l in link:
        links.append(l['href'])
    
    stocks = soup.findAll("p","product-stock-status")
    stoc = []
    for stock in stocks:
        if(str(stock).split(">",1)[1].split("<",1)[0] == "în stoc" or str(stock).split(">",1)[1].split("<",1)[0] == ""):
            stoc.append(2)
        elif(str(stock).split(">",1)[1].split("<",1)[0] == "stoc epuizat"):
            stoc.append(0)
        else:
            stoc.append(1)
    descs = []
    desc = soup.findAll("a", "product-title js-product-url")
    for description in desc:
        descs.append(description.get_text().strip())
    return preturi[:5], links[:5], stoc[:5], descs[:5]

def search_STRADAIT(URL):
    url = 'https://www.stradait.ro/Produse/Filtru/Cautare:'
    URL = url + URL
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')

    prices = soup.findAll("div","product-price")

    preturi = []
    for pret in prices:
        if len(pret.text) > 0:
            try:
                preturi.append(int(str(pret).split(">",1)[1].split("<",1)[0][:-4].replace('.','')[:-3]))
            except:
                continue
                
    links = []
    for tag in soup.findAll("div","image-product-grid img-responsive"):
        links.append("https://www.stradait.ro" + tag.find("a", href=True)['href'])
    
    stocks = soup.findAll("div","stockinfo")
    stoc = []
    for stock in stocks:
        if("in stoc" in str(stock).split(">",1)[1].split("<",1)[0].lower()):
            stoc.append(2)
        elif(str(stock).split(">",1)[1].split("<",1)[0] == "Stoc epuizat"):
            stoc.append(0)
        else:
            stoc.append(1)
    
    descs = []
    desc = soup.findAll("span",style=re.compile(r'color:black'))
    for description in desc:
        descs.append(description.get_text().strip())
    return preturi[:3], links[:3], stoc[:3], descs[:3]


def search_CEL(URL):
    url = 'https://www.cel.ro/cauta/'
    URL = url + URL
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')

    prices = soup.findAll("b",attrs={"productprice":True})

    preturi = []
    for pret in prices:
        if len(pret.text) > 0:
            try:
                preturi.append(int(str(pret).split(">",1)[1].split("<",1)[0].replace('.','')))
            except:
                continue
    links = []
    for tag in soup.findAll("div","productListing-poza"):
        if(tag.find("a", href=True)['href'].find("cel.ro") >= 0):
            links.append(tag.find("a", href=True)['href'])
        else:
            links.append("https://www.cel.ro" + tag.find("a", href=True)['href'])
            

    text = str(soup)[str(soup).find("try {$('div["):].split("\n")
    contor = 0
    stoc = []
    for line in text:
        if(contor == 3):
            break
        try:
            in_stoc = re.search(r".*class='info_stoc .*>(.*)<.*",line).group(1)
            if(in_stoc == "In stoc"):
                stoc.append(2)
            else:
                stoc.append(1)
            contor += 1
        except :
            pass
    descs = []
    desc = soup.findAll("a","productListing-data-b product_link product_name")
    for description in desc:
        descs.append(description.get_text().strip())
    return preturi[:3], links[:3], stoc[:3], descs[:3]



def search_MEDIAGALAXY(URL):
    url = 'https://cerberus.mediagalaxy.ro/catalog/search/'
    URL = url + URL
    page = requests.get(URL,timeout=100)

    soup = BeautifulSoup(page.content, 'html.parser')
    result = json.loads(str(soup))
    
    preturi = []
    links = []
    stoc = []
    descriptions = []
    for i in range(3):
        preturi.append(result['products'][i]['price'])
        links.append("https://mediagalaxy.ro/" + str(result['products'][i]['url_key']) + "/cpd/" + str(result['products'][i]['sku']))
        if(result['products'][i]['stock_status'] == 2):
            stoc.append(1)
        elif(result['products'][i]['stock_status'] == 1):
            stoc.append(2)
        elif(result['products'][i]['stock_status'] == 0):
            stoc.append(0)
        descriptions.append(result['products'][i]['name'])

    return preturi, links, stoc,descriptions

def search_ALTEX(URL):
    url = 'https://fenrir.altex.ro/catalog/search/'
    URL = url + URL
    page = requests.get(URL,timeout=100)

    soup = BeautifulSoup(page.content, 'html.parser')
    result = json.loads(str(soup))
    
    preturi, links, stoc, descriptions = [] , [], [], []
    for i in range(3):
        preturi.append(result['products'][i]['price'])
        links.append("https://altex.ro/" + str(result['products'][i]['url_key']))
        if(result['products'][i]['stock_status'] == 2):
            stoc.append(1)
        elif(result['products'][i]['stock_status'] == 1):
            stoc.append(2)
        elif(result['products'][i]['stock_status'] == 0):
            stoc.append(0)
        descriptions.append(result['products'][i]['name'])
    return preturi, links, stoc, descriptions

def search_VEXIO(URL):
    url = 'https://sb.searchnode.net/v1/query/docs?query_key=a3nmqeRLHM2AU656Z8CnKv0xCGnitxan&search_query='
    URL = url + URL
    page = requests.get(URL,timeout=100)

    soup = BeautifulSoup(page.content, 'html.parser')
    result = json.loads(str(soup))
    
    preturi = []
    links = []
    stoc = []
    descriptions = []
    for i in range(3) if len(result['docs']) >= 3 else range(len(result['docs'])):
        preturi.append(result['docs'][i]['f_price'])
        links.append("https://www.vexio.ro" + str(result['docs'][i]['url']))
        if(result['docs'][i]['s_stock_text'] == "contactati-ne pentru info stoc"):
            stoc.append(1)
        elif(result['docs'][i]['s_stock_text'] == "in stoc depozit"):
            stoc.append(2)
        else:
            stoc.append(0)
        descriptions.append(result['docs'][i]['s_title'])
            
    return preturi, links, stoc, descriptions


def search_OLX(url):
   
    prices = []
    links = []
    for i in range(1,5):
        URL = 'https://www.olx.ro/oferte/q-' + url +'/?page=' + str(i) +'&search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON'
        soup = get_html_source(URL)
        if((soup.findAll('input','br3')[0].get('value')) != url):
            break
        desc = soup.body.findAll(text="Asigura-te ca ai scris corect (se intampla oricui) sau incearca o cautare mai generala")
        if(len(desc) > 0):
            break
        price, link = get_prices_olx(soup)
        for pret in price:
            prices.append(pret)
        for lnk in link:
            links.append(lnk)
    return prices, links




def search_OLX_iphone11_11pro(url, which_phone):
    """ phone == iphone 11 ? which_phone = 1 : which_phone = 0"""
    model_nou = ModifyVGG(224,2)
    model_nou.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)
    model = nn.DataParallel(model_nou)
    model.load_state_dict(torch.load("trained_model"))
    
    prices = []
    links = []
    current_name = 0
    j=0
    for i in range(1,5):
        URL = 'https://www.olx.ro/oferte/q-' + url + '/?page=' + str(i) +'&search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON'
        soup = get_html_source(URL)
        current_name, price, link = download_images(soup,current_name)
        for pret in price:
            prices.append(pret)
        for lnk in link:
            links.append(lnk)
    i=0
    prices_final, links_final = [], []
    for file in os.listdir(IMAGE_LOCATION):
        if file.endswith('.jpg'):
            try:
                output = (model(image_loader(data_transforms, IMAGE_LOCATION + file))[0])
                if(F.softmax(output[0],dim=0)[which_phone] > 0.8):
                    prices_final.append(prices[int(os.path.splitext(file)[0])-1])
                    links_final.append(links[int(os.path.splitext(file)[0])-1])
            except:
                print("sss") 
        i += 1
    for file in os.listdir('testing'):
        if file.endswith('.jpg'):
            os.remove("testing/" + file)
    return prices_final, links_final, prices, links
    

In [5]:
def make_prices(siteid, prodtitle, function, lastprice):
    
    
    pret, link, stoc, description = function(prodtitle)
    prices = []


    descriptions = []
    for desc in description:   
        if (siteid == 2 or siteid == 6):
            if(desc.lower().find((prodtitle + ",").lower()) != -1):
                descriptions.append(desc)            
        else:
            if(desc.lower().find((prodtitle + " ").lower()) != -1 or desc.lower().find((prodtitle + ",").lower()) != -1):
                descriptions.append(desc)
    for desc in descriptions:
        p = pret[description.index(desc)]
        if(p / lastprice > 0.75):
            prices.append(p)
    if len(prices):
        price_update = (min(prices))
        link_update = (link[pret.index(min(prices))])
        stock_update = (stoc[pret.index(min(prices))])
        desc_update = (description[pret.index(min(prices))])
    else:
        price_update = (0)
        link_update = (0)
        stock_update = (0)
        desc_update = prodtitle
    return siteid, price_update, link_update, stock_update, desc_update

In [6]:
def update_prices(prodtitle, lastprice):
    """ Updateaza preturile de pe toate site-urile produsulului cu id-ul idprod"""
    update = []
    
    update.append([make_prices(2,prodtitle, search_ALTEX, lastprice)])
    update.append([make_prices(6,prodtitle, search_MEDIAGALAXY, lastprice)])
    update.append([make_prices(3,prodtitle, search_VEXIO, lastprice)])
    update.append([make_prices(5,prodtitle, search_STRADAIT, lastprice)])
    update.append([make_prices(4,prodtitle, search_CEL, lastprice)])
    update.append([make_prices(1,prodtitle, search_EMAG, lastprice)])
    return update

In [7]:
def update_prices_db(idprod, prodtitle, lastprice):
    """ Updateaza preturile din baza de date ale produslui idprod"""
    updates = update_prices(prodtitle, lastprice)
    if (idprod == 1):
        updates.append([7,np.mean([int(x) for x in search_OLX_iphone11_11pro('iphone 11 pro',0)[0]]), 'https://www.olx.ro/oferte/q-iphone-11-pro/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])
    elif (idprod == 18):
        updates.append([7,np.mean([int(x) for x in search_OLX_iphone11_11pro('iphone 11',1)[0]]), 'https://www.olx.ro/oferte/q-iphone-11/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])
    else:
        updates.append([7,np.mean([int(x) for x in search_OLX(prodtitle)[0]]), 'https://www.olx.ro/oferte/q-' + prodtitle +'/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])

    

    min_price = 99999
    for update in updates:
        if(update[0] != 7):
            if(min_price > round(update[0][1], 2) and round(update[0][1], 2) != 0):
                min_price = round(update[0][1], 2)
            sql = "UPDATE `new_schema`.`products_price` SET `availability` = '"+ str(update[0][3]) +"', `price` = '" + str((round(update[0][1], 2))) +"', `prodtitle` = '" + str(update[0][4]) +"', `link` = '" + str(update[0][2]) +"' WHERE (`idprod` = '" + str(idprod) +"' and `site` = '" + str(update[0][0]) +"');"
        else:
            olx_price = round(update[1], 2)
            sql = "UPDATE `new_schema`.`products_price` SET `availability` = '"+ str(update[3]) +"', `price` = '" + str((round(update[1], 2))) +"', `link` = '" + str(update[2]) +"' WHERE (`idprod` = '" + str(idprod) +"' and `site` = '" + str(update[0]) +"');"
        update_sql(sql)
    if(not (min_price > 1000 or min_price < 10000)):
        min_price = olx_price
    update_sql("UPDATE `new_schema`.`products` SET `price` = '" + str(min_price) + "' WHERE (`idproducts` = '" + str(idprod) +"');")
    return

In [8]:
def update_prices_db_first_time(idprod, prodtitle):
    """ Updateaza preturile din baza de date ale produslui idprod"""
    updates = update_prices(prodtitle, 1500)
    if (idprod == 1):
        updates.append([7,np.mean([int(x) for x in search_OLX_iphone11_11pro('iphone 11 pro',0)[0]]), 'https://www.olx.ro/oferte/q-iphone-11-pro/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])
    elif (idprod == 18):
        updates.append([7,np.mean([int(x) for x in search_OLX_iphone11_11pro('iphone 11',1)[0]]), 'https://www.olx.ro/oferte/q-iphone-11/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])
    else:
        updates.append([7,np.mean([int(x) for x in search_OLX(prodtitle)[0]]), 'https://www.olx.ro/oferte/q-' + prodtitle +'/?search%5Bfilter_float_price%3Afrom%5D=1500&currency=RON%27',2])

        
    
    

    for update in updates:
        if(update[0] != 7):
            sql = "INSERT INTO `new_schema`.`products_price` (`idprod`, `site`, `prodtitle`, `availability`, `price`, `link`) VALUES ('"+ str(idprod) +"', '"+ str(update[0][0]) +"', '"+ str(update[0][4]) +"', '"+ str(update[0][3]) +"', '" + str(round(update[0][1], 2)) +"', '" + str(update[0][2]) +"');"
        else:
            sql = "INSERT INTO `new_schema`.`products_price` (`idprod`, `site`, `prodtitle`, `availability`, `price`, `link`) VALUES ('"+ str(idprod) +"', '"+ str(update[0]) +"', '"+ str(prodtitle) +"', '"+ str(update[3]) +"', '" + str(round(update[1], 2)) +"', '" + str(update[2]) +"');"
        update_sql(sql)
    return

In [9]:
def update_sql(sql):
    db = MySQLdb.connect("DB_IP","DB_username","DB_pass","DB_schema" )
    cursor = db.cursor()


    try:
        cursor.execute(sql)
        db.commit()
    except ValueError:
        print("Nu am putut updata!")
        print(ValueError)
        db.rollback()
    db.close()
    return

In [10]:
def select_sql(sql):
    db = MySQLdb.connect("DB_ip","DB_username","DB_passDB_pass","DB_schema" )
    cursor = db.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    db.close()
    return results

In [11]:
def get_products_to_update_first_time():
    products_to_update = select_sql("SELECT * FROM new_schema.new_products;")
    last_id = int(select_sql("SELECT idproducts FROM new_schema.products ORDER BY idproducts DESC LIMIT 1;")[0][0])
    for product in products_to_update:
        try:
            last_id += 1
            update_prices_db_first_time(last_id, product[1])
            min_price = select_sql("SELECT MIN(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(last_id) + "';")[0][0]
            print("'"+ str(last_id) +"', '"+ str(min_price)+"', '"+ str(product[1])+"', '"+ str(product[2])+"'")
            update_sql("INSERT INTO `new_schema`.`products` (`idproducts`, `prodtitle`, `price`, `prodimg`, `maker`) VALUES ('"+ str(last_id) +"', '"+str(product[1]) +"', '"+ str(min_price)+"', '"+ str(product[2])+"', '"+ str(product[3])+"');")
            min_price = select_sql("SELECT MIN(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(last_id) + "' and site != 7;")[0][0]
            avg_price = select_sql("SELECT AVG(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(last_id) + "' and site != 7;")[0][0]
            if(min_price == 'None'):
                min_price = last_price
            if(avg_price == 'None'):
                avg_price = last_price
            update_sql("INSERT INTO `new_schema`.`products_price_history` (`idprod`, `date`, `min_price`, `avg_price`) VALUES ('"+ str(last_id) +"', "+ "curdate()" +", '"+ str(min_price) +"', '"+ str(avg_price) +"');")
        except:
            print("I'm going forward!")
    update_sql("DELETE FROM new_schema.new_products")

In [12]:
def get_products_to_update():
    products_to_update = select_sql("SELECT * FROM new_schema.products;")
    for product in products_to_update:
        print(product)
        try:
            if( product[2] == None):
                last_price = 1
            else:
                last_price = float(product[2])
            update_prices_db(product[0], product[1],last_price)
            min_price = select_sql("SELECT MIN(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(product[0]) + "' and site != 7;")[0][0]
            avg_price = select_sql("SELECT AVG(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(product[0]) + "' and site != 7;")[0][0]
            print("'"+ str(product[0]) +"', '"+ str(min_price)+"', '"+ str(product[1])+"', '"+ str(last_price)+"'")
            if(min_price == None):
                min_price = select_sql("SELECT MIN(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(product[0]) + "' and site = 7;")[0][0]
            if(avg_price == None):
                avg_price = select_sql("SELECT MIN(NULLIF(price, 0)) FROM new_schema.products_price WHERE idprod = '" + str(product[0]) + "' and site = 7;")[0][0]
            update_sql("INSERT INTO `new_schema`.`products_price_history` (`idprod`, `date`, `min_price`, `avg_price`) VALUES ('"+ str(product[0]) +"', "+ "curdate()" +", '"+ str(min_price) +"', '"+ str(avg_price) +"');")
        except:
            print("I'm going forward!")
        print("--------------------------------\n")

In [33]:
get_products_to_update_first_time()

'19', '2091.83', 'iPhone XR', 'https://s12emagst.akamaized.net/products/17043/17042909/images/res_beeb2a5b938cd2fc51ad063da79f4321_450x450_ckv0.jpg'


In [14]:
get_products_to_update()

(1, 'Apple iPhone 11 Pro Max', '5899', 'poze/iphone11promax.jpg', 'Apple')
'1', '5499', 'Apple iPhone 11 Pro Max', '5899.0'
--------------------------------

(2, 'Apple iPhone X', '4549.99', 'https://s12emagst.akamaized.net/products/8892/8891494/images/res_9902358bb73d36d099ea7afc6f48cd8b_full.jpg', 'Apple')
'2', '3769', 'Apple iPhone X', '4549.99'
--------------------------------

(9, 'Samsung Galaxy S20 Ultra', '5649', 'https://s12emagst.akamaized.net/products/30428/30427200/images/res_a2ebc264eb9d6279adaa3ca6e83a0e72_450x450_anfp.jpg', 'Samsung')
'9', '4499.9', 'Samsung Galaxy S20 Ultra', '5649.0'
--------------------------------

(10, 'Google Pixel 4 XL', '3420.99', 'https://s12emagst.akamaized.net/products/26604/26603266/images/res_98a7671962dc515a39732150a215bc15_450x450_b3pc.jpg', 'Google')
'10', '3414.99', 'Google Pixel 4 XL', '3420.99'
--------------------------------

(11, 'Huawei P30 Pro', '2699.9', 'https://s12emagst.akamaized.net/products/20936/20935294/images/res_ba09711c